In [1]:
import pandas as pd
import pandas as np
import os

In [2]:
path = r'/Users/sarahedens/Desktop/Instacart Basket Analysis'

In [4]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_merged.pkl'))

# 1a. Creating 'price_label' column

In [5]:
# Creating a subset of first one million rows
df = df_ords_prods_merge[:1000000]

In [6]:
df.shape

(1000000, 14)

In [7]:
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [8]:
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/zn/5_w_pldx4xx695bgyc2zgrrh0000gn/T/ipykernel_42870/2848743372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [9]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    652638
Low-range product    338018
High range             9344
Name: count, dtype: int64

In [10]:
df['prices'].max()

24.5

In [11]:
# Use loc() method on df subset to apply price range function.
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/zn/5_w_pldx4xx695bgyc2zgrrh0000gn/T/ipykernel_42870/3277917827.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'
/var/folders/zn/5_w_pldx4xx695bgyc2zgrrh0000gn/T/ipykernel_42870/3277917827.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'High-range product' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [12]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [13]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [14]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     652638
Low-range product     338018
High-range product      9344
Name: count, dtype: int64

In [15]:
# Use loc() method on entire ords_prods_merge dataframe to apply price range function.
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/zn/5_w_pldx4xx695bgyc2zgrrh0000gn/T/ipykernel_42870/1171807246.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'High-range product' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [16]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [17]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [18]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

# 1b. Create 'busiest_day' column

In [19]:
# Change "order_dow" column name to "order_day_of_week."
df_ords_prods_merge.rename(columns = {'order_dow' : 'orders_day_of_week'}, inplace = True)

In [20]:
# Find frequency of orders_day_of_week.

df_ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [21]:
# Use for-loop to determine whether orders are on "busiest day" (0 = Saturday), "least busy" (4 = Wednesday), or "regularly busy" (other days of the week).

result = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [22]:
# Create new column in ords_prods_merge to view results in context.

df_ords_prods_merge['busiest_day'] = result

In [23]:
df_ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

# 2. Update 'busiest_day' column to 'busiest days' to identify order day of the week as 'busiest days', 'slowest days', or 'regularly busy'.

In [24]:
# Create column “busiest_days" from "orders_day_of_week" to identify two busiest days (0, 1), slowest days (4, 3), or regularly busy (all other days).

result_2 = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0 or value == 1:
    result_2.append("Busiest days")
  elif value == 4 or value == 3:
    result_2.append("Slowest days")
  else:
    result_2.append("Regularly busy")

In [25]:
result_2

['Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',

# 3. Check values of new 'busiest days' column for accuracy with observations in markdown format.

In [26]:
# Create new 'busiest_days' column in ords_prods_merge to view results in context.

df_ords_prods_merge['busiest_days'] = result_2

In [27]:
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    12916111
Busiest days      11864412
Slowest days       7624336
Name: count, dtype: int64

In [28]:
# Check output of ords_prods_merge with new "busiest days" column.

df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,both,Mid-range product,Regularly busy,Slowest days
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Slowest days


Both columns representing the busiest days align with the total count recorded for the orders placed on each day of the week. For instance, in the column denoting multiple busy days, the days labeled as 3 and 4 are categorized as the slowest days. Consequently, when we combine the counts for both 3 and 4 (3840534 + 3783802), we obtain the total count for the slowest days, which is 7624336, consistent with the count in the busiest days column.

# 4. Create new column 'busiest_period_of_day' to identify time periods -- 'most orders', 'average orders', and 'fewest orders'.

In [29]:
# Change "time_of_day" column name to "order_hour_of_day."

df_ords_prods_merge.rename(columns = {'time_of_day' : 'order_hour_of_day'}, inplace = True)

In [30]:
# Check value counts in "order_hour_of_day" column.

df_ords_prods_merge['order_hour_of_day'].value_counts()

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

- Most orders = 10, 11, 14, 15, 13, 12, 16, and 9; 
- Fewest orders = 23, 6, 0, 1, 5, 2, 4, and 3; 
- Average orders = all remaining values in order_hour_of_day.

In [31]:
# Create column “busiest_period_of_day" from "order_hour_of_day" to identify "Most orders", "Fewest orders", or "Average orders".

result_3 = []

for value in df_ords_prods_merge["order_hour_of_day"]:
  if value in [10,11,14,15,13,12,16,9]:
    result_3.append("Most orders")
  elif value in [23,6,0,1,5,2,4,3]:
    result_3.append("Fewest orders")
  else:
    result_3.append("Average orders")

In [32]:
result_3

['Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most ord

In [33]:
# Create new column "busiest_period_of_day" in ords_prods_merge.

df_ords_prods_merge['busiest_period_of_day'] = result_3

# 5. Print frequency of 'busiest_period_of_day' column.

In [34]:
#Print value counts in "busiest_period_of_day" column.

df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21118071
Average orders     9997651
Fewest orders      1289137
Name: count, dtype: int64

# 6. Export dataframe as a pickle file to 'Prepared Data' folder.

In [36]:
#Exporting the dataframe; Using pickle format
df_ords_prods_merge.to_pickle(os.path.join(path,'02 Data','Prepared Data', 'orders_products_merged_derived.pkl'))